# Assignment 2: Forest Fires - Other Models

Forest fires dataset found here:
https://archive.ics.uci.edu/ml/datasets/forest+fires
Please place csv in route


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime
from sklearn.svm import SVR

#### Here we demonstate the hyperparameter selection and nested cross validation results for our models

# Preprocessing

First we set a reproducible seed and separate out the target feature 

In [48]:
np.random.seed()

raw = pd.read_csv('forestfires.csv')

# subset X columns and Y column
X = raw.iloc[:, 0:12]
y = np.array(raw.iloc[:, 12])

We then perform the sine cosine feature transformation of the month and day features by first mapping them to sequential numerics and then generating the following features for each of them:
$$x_{\sin }=\sin \left(\frac{2 * \pi * x}{\max (x)}\right)$$

$$x_{\cos }=\cos \left(\frac{2 * \pi * x}{\max (x)}\right)$$

In [49]:


# cos sin transform categorical sequential features
# map features to numerics
monthDict = dict(zip(['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'],[1,2,3,4,5,6,7,8,9,10,11,12]))
dayDict = dict(zip(['mon','tue','wed','thu','fri','sat','sun'],[1,2,3,4,5,6,7]))

X['month'] = X['month'].map(monthDict).astype(float)
X['day'] = X['day'].map(dayDict).astype(float)


# map each cyclical variable onto a circle so lowest value for that variable appears right next to the largest value.
X['day_sin'] = np.sin(X.day*(2.*np.pi/7))
X['day_cos'] = np.cos(X.day*(2.*np.pi/7))
X['mnth_sin'] = np.sin(X.month*(2.*np.pi/12))
X['mnth_cos'] = np.cos(X.month*(2.*np.pi/12))

Next we drop the original month and day features and convert the ints to floats

In [50]:
# drop original categorical variables
X = X.drop(['month', 'day'], 1)

# fix int warning
X['X'] = X['X'].astype(float)
X['Y'] = X['Y'].astype(float)
X['RH'] = X['RH'].astype(float)

# Modelling

We then define the negative log likelihood function as follows:
$$N N L=-\log p\left(y_{*} | D, x_{*}\right)=\frac{1}{2} \log \left(2 \pi \sigma_{*}^{2}\right)+\frac{\left(y_{*}-\bar{f}\left(x_{*}\right)\right)^{2}}{2 \sigma_{*}^{2}}$$

In [51]:
# define negative log likelihood of sample
def negative_log_likelihood(y, p):
    result = 0.5 * np.log(2 * np.pi * np.var(p)) + (((y - np.mean(p)) ** 2) / (2 * np.var(p)))
    return result


The feature sets to tune over in the inner fold are then defined. We utilise the four sets from the original paper and the additional complete set of all features: *STFWIM*

In [52]:
#feature selection list
STFWIM = ['X', 'Y', 'FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain',
       'day_sin', 'day_cos', 'mnth_sin', 'mnth_cos']

STFWI = ['X', 'Y', 'FFMC', 'DMC', 'DC', 'ISI', 'day_sin', 'day_cos', 'mnth_sin', 'mnth_cos']

STM = ['X', 'Y', 'temp', 'RH', 'wind', 'rain', 'day_sin', 'day_cos', 'mnth_sin', 'mnth_cos']

FWI = ['FFMC', 'DMC', 'DC', 'ISI']

M = ['temp', 'RH', 'wind', 'rain']

In this step we define the nested cross validation function. 

First, a pipeline of the preprocessing steps is created to be reused in all folds. We then explicitly set the 10 outer and inner folds with a seed for reproducibility. 
We utilise sklearns GridSearchCV for the hyperparamater search of the inner fold, and take advantage of its multithreading to speed up the process

The list of outer fold reuslts to persist, inner fold results to pass to the outer fold and feature spaces to tune are then created. 
The outer loop splits test and train data, sending the train data into the inner loop for CV. The inner loop fits and validates models on the train folds of the outer kv whilst looping through the feature sets. The best of the models, evaluated by the argmax NMSE is then persisted, with its parameter values and feature set saved to be fitted on train and evaluated on outer test.

Score metrics and output results are then retrieved from the outer fold and persisted

In [53]:


def nested_crossval(model , parameters):
    # build pipeline
    pipeline = Pipeline([('scaler', MinMaxScaler()),
                         ('estimator',
                          TransformedTargetRegressor(regressor=model
                                                     , func=np.log1p, inverse_func=np.expm1))])

    # define outer and inner folds
    outer_kv = KFold(n_splits=10, shuffle=True, random_state=42)
    inner_kv = KFold(n_splits=10, shuffle=True, random_state=42)
    
    #instantiate inner CV grid search 
    cv = GridSearchCV(estimator=pipeline, param_grid=parameters, cv=inner_kv,
                      scoring="neg_mean_squared_error", n_jobs=-1, verbose=True)
    
    #create list of feature spaces to search and list of result to persist
    feature_space = [STFWIM, STFWI, STM, FWI, M]
    inner_result = []
    outer_result = []
    saving_inner_results = []
    i = 0
    for train, test in outer_kv.split(X):
        print("run", i)
        # loop over feature space using only training data and cross validator hyper param tuner
        for f in feature_space:
            cv.fit(X.loc[X.index[train], f], y[train])
            # persist models to fit best on training set
            inner_result.append([cv, cv.best_params_, f, cv.best_score_])
            print(f)
            print(cv.best_score_)
        # persits and reset inner result for next fold
        inner_df = pd.DataFrame(inner_result)
        # reset inner result
        inner_result = []
        # receive best model of run to fit on test set
        best_params_arg = inner_df.loc[:, 3].argmax()
        best_params = inner_df.iloc[best_params_arg, :]
        # fit best cv model hyper parameters on best feature set for that fold
        bcv = best_params[0]
        bfs = best_params[2]
        bcv.fit(X.loc[X.index[train], bfs], y[train])

        # get training/val and test scores
        train_score = best_params[3]
        test_score = bcv.score(X.loc[X.index[test], bfs], y[test])

        # get predictions and retrieve nll of test folds
        y_preds = cv.predict(X.loc[X.index[test], bfs])
        mae = mean_absolute_error(y[test], y_preds)
        nllval = negative_log_likelihood(y[test], y_preds)
        mean_nll = np.mean(nllval)

        outer_result.append([i, train_score, test_score, mae, bfs, best_params[1], y[test], nllval, mean_nll])
        i += 1

    testing = pd.DataFrame(outer_result)
    testing.columns = ['fold_number', 'train_nmse', 'test_nmse', 'test_mae', 'best_feature_set', 'best_hyperparams',
                       'test_set', 'nll', 'mean_nll']

    return testing


# Results

In reporting results we compare the unbiased average test fold performane compared to benchmarks. Due to the small sample size of the dataset we report that no one set of parameters and features were chosen in every outer fold for each model. We fit the majority chosen parameters here, but allow for feature set selection to illustrate the nested CV.

Note: The reported mean absolute deviation benchmark is actually a measurement of means absolute error

The Linear Regression was fitted with no regularisation as the MSE reducing result in all folds

In [61]:
start=datetime.now()

lr_results = nested_crossval(SGDRegressor(max_iter=5, tol=-np.infty, random_state=42),
                              [{"estimator__regressor__penalty": [None]}
                               #{"estimator__regressor__penalty": ['l2', 'l1'],
                               #"estimator__regressor__alpha": [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]},
                               #{"estimator__regressor__penalty": ['elasticnet'],
                                #"estimator__regressor__alpha": [1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
                                #"estimator__regressor__l1_ratio": [0.001, 0.25, 0.5, 0.75, 0.999]}
                              ])

lrruntime = datetime.now()-start
print("finished")

run 0
Fitting 10 folds for each of 1 candidates, totalling 10 fits
['X', 'Y', 'FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain', 'day_sin', 'day_cos', 'mnth_sin', 'mnth_cos']
-2056.669976955912
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent 

['X', 'Y', 'FFMC', 'DMC', 'DC', 'ISI', 'day_sin', 'day_cos', 'mnth_sin', 'mnth_cos']
-2056.9580669441184
Fitting 10 folds for each of 1 candidates, totalling 10 fits
['X', 'Y', 'temp', 'RH', 'wind', 'rain', 'day_sin', 'day_cos', 'mnth_sin', 'mnth_cos']
-2057.9808771355692
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\ipykernel_launcher.py:38: FutureWarning: 
The current behaviour of 'Series.argmax' is 

['FFMC', 'DMC', 'DC', 'ISI']
-2056.227030703049
Fitting 10 folds for each of 1 candidates, totalling 10 fits
['temp', 'RH', 'wind', 'rain']
-2057.6514778380238
Fitting 10 folds for each of 1 candidates, totalling 10 fits


...

[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.


['X', 'Y', 'FFMC', 'DMC', 'DC', 'ISI', 'day_sin', 'day_cos', 'mnth_sin', 'mnth_cos']
-4515.026836187481
Fitting 10 folds for each of 1 candidates, totalling 10 fits
['X', 'Y', 'temp', 'RH', 'wind', 'rain', 'day_sin', 'day_cos', 'mnth_sin', 'mnth_cos']
-4514.832722516506
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\ipykernel_launcher.py:38: FutureWarning: 
The current behaviour of 'Series.argmax' is 

['FFMC', 'DMC', 'DC', 'ISI']
-4514.474697515994
Fitting 10 folds for each of 1 candidates, totalling 10 fits
['temp', 'RH', 'wind', 'rain']
-4515.0155815530825
Fitting 10 folds for each of 1 candidates, totalling 10 fits
finished


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [62]:
#printing results
print("average LR test fold RMSE (Benchmark: 63.7)", np.sqrt(np.mean(np.negative(lr_results['test_nmse']))))
print("average LR test fold MSE:",np.mean(np.negative(lr_results['test_nmse'])))
print("average LR test fold MAE/'MAD' (Benchmark:12.71):", np.mean(lr_results['test_mae']))
print("average LR test fold NLL:", np.mean(lr_results['mean_nll']))
print("LR Fitting Runtime:" ,lrruntime)

average LR test fold RMSE (Benchmark: 63.7) 64.51391347829868
average LR test fold MSE: 4162.045032285407
average LR test fold MAE/'MAD' (Benchmark:12.71): 12.972990752106409
average LR test fold NLL: 17838.233357842262
LR Fitting Runtime: 0:00:07.818104


The Random forest had a max depth of None in 6 folds out of 10 folds and a min sample split of 2 in 9 out 10 folds 

In [ ]:
start=datetime.now()
rf_results = nested_crossval(RandomForestRegressor(random_state=42),
                       {"estimator__regressor__n_estimators": [500],
              "estimator__regressor__max_depth": [ None],
             "estimator__regressor__min_samples_split": [2]
              })

rfruntime = datetime.now()-start
print("finished")

run 0
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    2.6s finished
C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


['X', 'Y', 'FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain', 'day_sin', 'day_cos', 'mnth_sin', 'mnth_cos']
-2060.2166196499566
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    1.7s finished
C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


['X', 'Y', 'FFMC', 'DMC', 'DC', 'ISI', 'day_sin', 'day_cos', 'mnth_sin', 'mnth_cos']
-2085.0179538321104
Fitting 10 folds for each of 1 candidates, totalling 10 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.7s finished
C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


['X', 'Y', 'temp', 'RH', 'wind', 'rain', 'day_sin', 'day_cos', 'mnth_sin', 'mnth_cos']
-4608.504474813842
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.5s finished
C:\Users\SwagDaddy\Anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


['FFMC', 'DMC', 'DC', 'ISI']
-4510.462775207214
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.


In [9]:
#printing results
print("average RF test fold RMSE (Benchmark: 63.7)", np.sqrt(np.mean(np.negative(rf_results['test_nmse']))))
print("average RF test fold MSE:",np.mean(np.negative(rf_results['test_nmse'])))
print("average RF test fold MAE/'MAD' (Benchmark:12.71):", np.mean(rf_results['test_mae']))
print("average RF test fold NLL:", np.mean(rf_results['mean_nll']))
print("RF Fitting Runtime:" ,rfruntime)

NameError: name 'rf_results' is not defined

# Best Model

For the best overall performance we submit the SVM with a linear kernel, C set to 1000 and feature set *STFWIM* . It has close performance to both the 'MAD' and RMSE benchmark (even though we evalute on unseen data), and also the second best average NLL. 

This model fitting in best model notebook